<a href="https://colab.research.google.com/github/MYon003/Myon/blob/main/Projects/Michael_Bach_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Loading the bach.csv into a pandas table
import pandas as pd

bach = pd.read_csv('bach.csv')
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [2]:
# Make bass into usable data for the classifier
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

bach['bass'] = le.fit_transform(bach['bass'])
bach['bass'].unique()

array([12, 10,  7,  0,  4,  5, 14,  3, 13,  6, 11,  8, 15,  1,  2,  9])

In [3]:
# Turn data into usable data
one_hot = pd.get_dummies(bach[['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']], drop_first = True)
one_hot

,C_YES,C#_YES,D_YES,D#_YES,E_YES,F_YES,F#_YES,G_YES,G#_YES,A_YES,A#_YES,B_YES
0,1,0,0,0,0,1,0,0,0,1,0,0
1,1,0,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,0,0
3,1,0,0,0,0,1,0,0,0,1,0,0
4,1,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,1,0,0,0,0,1,0,0,1,0
5661,0,0,1,0,0,0,0,1,0,1,0,0
5662,1,0,0,0,1,0,0,1,0,0,0,0
5663,1,0,0,0,1,0,0,1,0,0,1,0


In [4]:
# Drop the useless columns
bach = bach.drop(['event_number', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'], axis = 1)

In [5]:
# Join the one hot with the main data
bach = bach.join(one_hot)
bach

,choral_ID,bass,meter,chord_label,C_YES,C#_YES,D_YES,D#_YES,E_YES,F_YES,F#_YES,G_YES,G#_YES,A_YES,A#_YES,B_YES
0,000106b_,12,3,F_M,1,0,0,0,0,1,0,0,0,1,0,0
1,000106b_,10,5,C_M,1,0,0,0,1,0,0,1,0,0,0,0
2,000106b_,10,2,C_M,1,0,0,0,1,0,0,1,0,0,0,0
3,000106b_,12,3,F_M,1,0,0,0,0,1,0,0,0,1,0,0
4,000106b_,12,2,F_M,1,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,14,4,G_m,0,0,1,0,0,0,0,1,0,0,1,0
5661,015505b_,14,3,G_m,0,0,1,0,0,0,0,1,0,1,0,0
5662,015505b_,5,5,C_M,1,0,0,0,1,0,0,1,0,0,0,0
5663,015505b_,5,3,C_M,1,0,0,0,1,0,0,1,0,0,1,0


In [6]:
# The provided train - test set
bach.set_index('choral_ID', inplace=True)
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']

from sklearn.model_selection import train_test_split

bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)

In [7]:
# Normalizing Data
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

bach_train_features = mms.fit_transform(bach_train_features)
bach_test_features = mms.fit_transform(bach_test_features)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

kNN = KNeighborsClassifier(n_neighbors = 2, weights = 'distance', leaf_size = 30)

subspace_clf = BaggingClassifier(kNN, n_estimators=200, max_samples=.7,
                                 max_features=.8, bootstrap=True, n_jobs=-1, random_state=45)

subspace_clf.fit(bach_train_features, bach_train_labels)

predictions = subspace_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7466902030008826